In [1]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, KeywordsOptions, SummarizationOptions, EntitiesOptions
from gensim.summarization.summarizer import summarize
import requests
from bs4 import BeautifulSoup
import traceback

In [2]:
urlList = [ #Input URL inside '' and add comma(,) except for the last element
    #Hong Kong
    'https://www.dlapiperdataprotection.com/index.html?t=law&c=HK',
    'https://www.hugillandip.com/2021/07/hong-kong-lawyer-hong-kong-court-warns-an-application-based-on-data-obtained-through-backdoors-is-an-affront-to-the-court/', 
    'https://www.hugillandip.com/2021/06/lexisnexis-penalties-for-data-privacy-breaches/', 
    'https://www.onc.hk/en_US/publication/hong-kong-amends-personal-data-protection-laws-to-combat-doxxing', 
    'https://www.aohub.com/aohub/publications/hong-kongs-personal-data-law-given-new-teeth-what-they-mean-for-businesses?nav=FRbANEucS95NMLRN47z%2BeeOgEFCt8EGQ71hKXzqW2Ec%3D&key=BcJlhLtdCv6%2FJTDZxvL23TQa3JHL2AIGr93BnQjo2SkGJpG9xDX7S2thDpAQsCconWHAwe6cJTkvxGr0G7eHl7pskrooXVOA&uid=MBqVugGtm%2BY%3D', 
    'https://globalinvestigationsreview.com/insight/know-how/data-privacy-and-transfer-in-investigations/report/hong-kong', 
    'https://www.clydeco.com/en/insights/2021/12/hong-kong-s-personal-data-protection-reforms', 
    'http://www.hk-lawyer.org/content/personal-data-privacyamendment-ordinance-2021-new-regulatory-regime', 
    'http://www.hk-lawyer.org/content/european-commission%E2%80%99s-new-standard-contractual-clauses-international-data-transfers', 
    'https://www.tannerdewitt.com/personal-data-privacy/', 
    'https://www.onc.hk/en_US/publication/data-privacy-for-banks-and-financial-institutions', 
    'https://www.engage.hoganlovells.com/knowledgeservices/news/the-cathay-pacific-breach-is-data-protection-and-cyber-security-law-in-hong-kong-about-to-receive-an-upgrade_2', 
    'https://news.bloomberglaw.com/tech-and-telecom-law/transunion-breached-hong-kong-data-security-law-regulator-says', 
    'https://www.sia-partners.com/en/news-and-publications/from-our-experts/state-ethical-ai-hong-kong',
    'https://www.mondaq.com/hongkong/data-protection/1134772/hong-kong-issues-guidance-on-the-use-of-ai#:~:text=The%20Guidance%20further%20recommends%20that,irrelevant%20to%20the%20purposes%20concerned',
    'https://www.pcpd.org.hk/english/news_events/media_statements/press_20210818.html',
    'https://www.dataguidance.com/notes/hong-kong-data-protection-overview#:~:text=486',
    'https://www.gibsondunn.com/sfc-and-hkma-introduce-requirements-for-hong-kong-licensed-firms-and-banks-dealing-with-virtual-assets-and-virtual-asset-service-providers/',
    'https://www.legco.gov.hk/research-publications/english/essentials-1718ise08-e-wallets-in-hong-kong.htm#:~:text=The%20Ordinance%20stipulates%20that%20personal,or%20a%20directly%20related%20purpose',
    'https://uk.practicallaw.thomsonreuters.com/w-034-3579?transitionType=Default&contextData=(sc.Default)&firstPage=true',    
    
    #CHina
    'https://www.dlapiperdataprotection.com/index.html?t=law&c=CN&c2=',
    'https://www.dataguidance.com/notes/china-data-protection-overview', 
    'https://www.skadden.com/Insights/Publications/2021/11/Chinas-New-Data-Security-and-Personal-Information-Protection-Laws', 
    'https://www.whitecase.com/publications/alert/china-personal-information-protection-law-will-become-effective-soon', 
    'http://www.hk-lawyer.org/content/new-era-personal-information-protection-mainland-china', 
    'http://www.hk-lawyer.org/content/cross-border-transfer-data-under-personal-information-protection-law-mainland', 
    'http://www.hk-lawyer.org/content/why-hong-kong-must-stay-ahead-changing-china-privacy-laws', 
    'http://www.deheheng.com/Archives/IndexArchives/index/a_id/8030.html', 
    'http://www.li-partners.com/en/index.php?s=/sys/833.html', 
    'http://www.zhonglun.com/Content/2021/08-21/0130117987.html', 
    'http://www.zhonglun.com/Content/2018/04-27/1424442402.html', 
    'https://www.deacons.com/sc/2021/08/27/legal-update-on-the-new-requirements-for-employers-under-the-personal-information-protection-law-in-china/', 
    'https://www.jingtian.com/Content/2021/08-25/1716461261.html', 
    'http://www.jiayuan-law.com/cn/news_content.aspx?Lan=CN&MenuID=00000000000000000006&KeyID=00000000000000002065&Type=00000000000000000081', 
    'https://www.globallegalinsights.com/practice-areas/ai-machine-learning-and-big-data-laws-and-regulations/china',
    'https://www.forrester.com/blogs/navigate-chinas-new-data-and-ai-regulations/',
    'https://www.mondaq.com/china/security/1158996/china-releases-draft-guidelines-to-help-organisations-identify-critical-data-under-the-data-security-law',
    #India
    'https://www.dlapiperdataprotection.com/index.html?t=law&c=IN&c2=',
    'https://www.dataguidance.com/notes/india-data-protection-overview', 
    'https://www.linklaters.com/en-hk/insights/data-protected/data-protected---india', 
    'https://iapp.org/news/a/a-look-at-proposed-changes-to-indias-personal-data-protection-bill/', 
    'https://www.mondaq.com/india/privacy-protection/1146570/update-on-data-protection-law', 
    'https://www.mondaq.com/india/privacy-protection/1146570/update-on-data-protection-law',
    'https://www.natlawreview.com/article/privacy-data-protection-capsule-india-s-turn-world-stage', 
    'https://www.natlawreview.com/article/data-privacy-standards-issued-india-legal-compliance-or-new-brand-differentiator',
    'https://portswigger.net/daily-swig/indias-personal-data-privacy-bill-what-does-it-mean-for-individuals-and-businesses', 
    'https://privacylibrary.ccgnlud.org/case/justice-ks-puttaswamy-ors-vs-union-of-india-ors'
    'https://www.natlawreview.com/article/regulations-e-wallets-gift-cards-and-vouchers-given-facelift',
    'https://cleartax.in/s/payment-wallet-license',
    'https://www.mondaq.com/india/fin-tech/1164634/virtual-digital-assets-in-india-a-sneak-peek-into-india39s-regulatory-framework',
    'https://indiaai.gov.in/research-reports/artificial-intelligence-and-privacy'
]

In [3]:
#Print metadata, keyword, category, entity and summary for each URL
#Result printed in reversed order as notion web clipper puts the lastly added URL on the top
#Note: Some URLs may throw exception such as "Access forbidden by target server". This case, processing is blocked by the website.
count = 0
for URL in urlList:
    count +=1
    print('<',count,'>')
    print(URL)
    
    authenticator = IAMAuthenticator('Suk_Yjz4CzP71YD4U_HcEGN0q76a24ZRODrz-rXVORPJ')
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2022-04-07',
        authenticator=authenticator
    )

    natural_language_understanding.set_service_url('https://api.kr-seo.natural-language-understanding.watson.cloud.ibm.com/instances/0de8d00e-0a41-4121-ac07-5264f62c77c2')

    try:
        #Metadata - title, Publication date
        metadata = natural_language_understanding.analyze(
            url=URL,
            features=Features(metadata={})).get_result()
        print('[Metadata -title, publication date]')
        print(metadata.get("metadata").get("title"))
        print(metadata.get("metadata").get("publication_date"))
        print('')

        #Keyword - top 10
        keyword = natural_language_understanding.analyze(
            language='en',
            url=URL,
            features=Features(keywords=KeywordsOptions(limit=10))).get_result()

        print('[Keyword - top 10]')
        for i in range(len(keyword.get("keywords"))):
            print(keyword.get("keywords")[i].get("text"))
        print('')

        #Category - top 3
        category = natural_language_understanding.analyze(
            url= URL,
            features=Features(categories=CategoriesOptions(limit=3))).get_result()
        print('[Category - top3]')
        for i in range(len(category.get("categories"))):
            print(category.get("categories")[i].get("label"))
        print('')

        #Entities 
        entity = natural_language_understanding.analyze(
            url=URL,
            features=Features(entities=EntitiesOptions(limit=3))).get_result()
        print('[Entities]')
        for i in range(len(entity.get("entities"))):
            print(entity.get("entities")[i].get("type") + ":", end = ' ')
            print(entity.get("entities")[i].get("text"))
        
        #summary 
        print()
        html = requests.get(URL)
        soup = BeautifulSoup(html.text)
        text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
        print("summary: ")
        print(summarize(text, word_count=100))
    except:
        traceback.print_exc()    
    #finish one url
    print('')
    print('############################################')
    print('')

< 1 >
https://www.dlapiperdataprotection.com/index.html?t=law&c=HK
[Metadata -title, publication date]
Law in Hong Kong, SAR - DLA Piper Global Data Protection Laws of the World
2021-12-22T00:00:00

[Keyword - top 10]
handling of personal data
data users
direct marketing
processing of the data
Hong Kong model
Personal Data
definition of personal data
data subject
classes of data users
separate concept of sensitive personal data

[Category - top3]
/news and politics/law
/technology & computing/computing/information and network security
/news and politics/crime

[Entities]
Organization: PCPD
Organization: Amendment Ordinance
Location: Hong Kong

summary: 
Nevertheless, non-binding best practice guidance published by the PCPD encourages compliance with the cross-border transfer restrictions in section 33 of the Ordinance, which prohibit the transfer of personal data to a place outside Hong Kong unless certain conditions are met (including a white list of jurisdictions; separate and volunt

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 60] Operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py", line 386, in _make_reque


############################################

< 17 >
https://www.dataguidance.com/notes/hong-kong-data-protection-overview#:~:text=486
[Metadata -title, publication date]
Hong Kong - Data Protection Overview
2021-11-17T00:00:00

[Keyword - top 10]
use of personal data
set of data protection principles
data user
provision of personal data
data subject
data processors
data users
Personal Data
purpose of these amendments
processing of personal data

[Category - top3]
/news and politics/law
/technology & computing/computing/information and network security

[Entities]
Organization: PDPO
Organization: PCPD
Organization: Amendment Ordinance

summary: 
Its main duty, as stated on its website, is to 'ensure the protection of the privacy of individuals in relation to personal data through monitoring and supervising compliance with the Ordinance, enforcing its provisions and promoting the culture of protecting and respecting personal data.' Under Section 38 of the PDPO, the PCPD has the power t

Traceback (most recent call last):
  File "<ipython-input-3-ec45b424fb9a>", line 63, in <module>
    print(summarize(text, word_count=100))
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gensim/summarization/summarizer.py", line 428, in summarize
    raise ValueError("input must have more than one sentence")
ValueError: input must have more than one sentence


[Metadata -title, publication date]
法律实务 |《个人信息保护法》的要点梳理及合规建议 | LI & PARTNERS－李偉斌律師行
2022-03-02T00:00:00

[Keyword - top 10]
日，十三届全国人大常委会第三十次会议表决通过了
日起施行，实践中直接援引
谢幸殷                         Category
日发布的
文铭
JR
PARTNERS
Li
rights
Research

[Category - top3]
/business and finance/industries/legal services industry
/news and politics/law
/personal finance/consumer banking

[Entities]
Number: 一
Number: 二
Number: 四

summary: 
(Available in Chinese Only) 随着信息化社会的不断发展，个人信息保护成为了社会各界重点关注的问题。2018年9月，关于个人信息保护的法律正式被纳入全国人民代表大会的立法计划，历时近三年，2021年8月20日，十三届全国人大常委会第三十次会议表决通过了《中华人民共和国个人信息保护法》（以下简称“《个人信息保护法》”或“该法”），该法自2021年11月1日起施行。 《个人信息保护法》分为八章共七十四条，明晰了个人信息处理活动中的相关主体的权利义务边界。该法是我国首部明确规定个人信息处理规则的法律，区别于之前分散颁布的部门规章，其较高的位阶反映了该法所调整的法律关系之重要性。该法对违法处理个人信息的处罚作出了严厉的规定，全面提升了违法违规成本，除了民事责任外，相关的处罚包括责令改正、给予警告，没收违法所得，责令暂停或者终止提供服务、罚款等行政责任，以及刑事责任。其中，针对情节严重的违法处理个人信息行为，可处五千万元以下或者上一年度营业额百分之五以下罚款；该项用营业额进行处罚的规定是继《中华人民共和国反垄断法》之后，我国第二部涉及按营业收入百分比进行行政处罚的法律，可对个人信息处理者起到较强的震慑作用。 本文将简要梳理《个人信息保护法》的要点，以为个人信息处理者提供若干合规参考。 一、《个人信息保护法》的部分要点梳

In [5]:
#For PDF documents:
TEXT = '''CMS Expert Guide: Data Law Navigator
Data protection and cybersecurity laws in Hong Kong
1 | Data protection and cybersecurity laws in Hong Kong
1.1 Local data protection laws and scope
1.2 Data protection authority
1.3 Anticipated changes to local laws
1.4 Sanctions & non-compliance
1.5 Registration / notification / authorisation
1.6 Main obligations and processing requirements
1.7 Data subject rights
1.8 Processing by third parties
1.9 Transfers out of country
1.10 Data Protection Officer
1.11 Security
1.12 Breach notification
1.13 Direct marketing
1.14 Cookies and adtech
1.15 Risk scale
1.16 Useful links
1 Data protection
2.1 Local cybersecurity laws and scope
2.2 Anticipated changes to local laws
2.3 Application
2.4 Authority
2.5 Key obligations
2.6 Sanctions & non-compliance
2.7 Is there a national computer emergency response team (CERT) or computer
security incident response team (CSIRT)?
2.8 National cybersecurity incident management structure
2.9 Other cybersecurity initiatives
2.10 Useful links
2 Cybersecurity
Table of contents
2 | Data protection and cybersecurity laws in Hong Kong
Data protection
1. Local data protection laws and scope
The Personal Data (Privacy) Ordinance(Cap. 486) (the "PDPO") is a comprehensive set of laws that is
technology-neutral and provides a set of Data Protection Principles outlining how data users should collect,
handle and use personal data.
2. Data protection authority
The Office of the Privacy Commissioner for Personal Datawww.pcpd.org.hk
3. Anticipated changes to local laws
The legislation is due for amendment since its last substantive amendment in 2012.
The Constitutional and Mainland Affairs Bureau releasedLC Paper. No. CB(2) 512/19-20(03), a discussion
paper seeking the Legislative Council’s Panel on Constitutional Affairs’ (the Panel) views on proposed changes
to the Personal Data (Privacy) Ordinance (Cap.486). The proposed changes follow proposals by the Privacy
Commissioner for Personal Data, and include six proposed amendments:
Inclusion of a Mandatory Data Breach Notification Mechanism;
Requirement for retention policy and specified Data Retention Period;
Provision of Sanctioning Powers to PCPD to impose administrative fines and raise relevant
criminal fine levels;
Regulation of Data Processors;
Amending the Definition of Personal Data to cover information relating to an "identifiable" natural
person;
Regulation of Disclosure of Personal Data of Other Data Subjects to curb doxing;
4. Sanctions & non-compliance
Administrative sanctions:
N/A
Criminal sanction s:
A summary of various offences and penalties under the Ordinance can be found
at: https://www.pcpd.org.hk/misc/files/table2_e.pdf
Others:
N/A
3 | Data protection and cybersecurity laws in Hong Kong
5. Registration / notification / authorisation
There is no requirement for notification/registration/authorisation for processing personal data (i.e. no
mechanism similar to that in UK Notification to process personal data - GOV.UK (www.gov.uk))
6. Main obligations and processing requirements
Data users shall comply with the six principles set out in Schedule 1 to the Ordinance:
personal data shall only be collected for a lawful purpose directly related to a function or activity
of the data user. The data collected should be necessary and adequate but not excessive for such
purpose. The means of collection should be lawful and fair;
data users are required to take all practicable steps to ensure that personal data is accurate and
not kept longer than is necessary for the fulfilment of the purpose for which the data is used. If
data users engage a data processor for handling personal data of other persons, data users should
adopt contractual or other means to ensure that the data processor comply with the mentioned
retention requirement;
data users shall not use personal data for any new purpose which is not or is unrelated to the
original purpose when collecting the data, unless with the data subject’s express and voluntary
consent;
data users shall take all practicable steps to protect the personal data they hold against
unauthorised or accidental access, processing, erasure, loss or use;
data users are required to take all practicable steps to ensure openness of their personal data
policies and practices, the kind of personal data held and the main purposes for holding it; and
data users shall provide data subjects with the right to request access to and correction of their
own personal data.
7. Data subject rights
Data subjects are given the right to access and make correction to their data.
8. Processing by third parties
No direct regulation on data processors. However, data are required to adopt contractual means to ensure
that data processors or sub-contractors adopt measures to ensure the safety of personal data.
9. Transfers out of country
There are currently no prescribed laws restricting transfer out as the relevant provision in the PDPO, section
33, is not yet in effect. Section 33 provides:
the place is specified by the Commissioner by notice in the Gazette that there is in force any law
which is substantially similar to, or serves the same purposes as, the Ordinance – no place has
satisfied this condition up to date.
The data user has reasonable grounds for believing that there is in force in that place any law
4 | Data protection and cybersecurity laws in Hong Kong
which is substantially similar to, or serves the same purposes as, the Ordinance;
The data subject has consented in writing to the transfer;
The data user has reasonable grounds for believing that the transfer is for the avoidance or
mitigation of adverse action against the data subject; it is not practicable to obtain the consent in
writing of the data subject to that transfer; but if it was practicable, such consent would be given;
The data is exempt from Data Protection Principle 3 by virtue of an exemption under Part VIII of
the Ordinance (such as personal data held for news activities, for domestic use, for purpose of
prevention of crime etc.); or
The data user has taken all reasonable precautions and exercised all due diligence to ensure that
the data will not, in that place, be collected, held, processed, or used in any manner which, if that
place were Hong Kong, would be a contravention of a requirement under the Ordinance. Please
note that use of recommended model data transfer clauses to develop an enforceable data
transfer contract by data users is one method to satisfy the required due diligence requirement.
10. Data Protection Officer
N/A
11. Security
There is no mandatory requirement. However, it is required that a data subject is informed of the name or
job title, and address, of the individual who is to handle the data access or correction request made to the
data user.
12. Breach notification
There is no mandatory requirement, but a data breach may amount to a contravention of
Data Protection Principle 4(1); and in Schedule 1 of the Ordinance;
The following action plan is recommended as practice to be adopted by data users:
immediate gathering of essential information relating to the breach;
contacting the interested parties and adopting measures to contain the breach;
assessing the risk of harm;
considering the giving of data breach notification: notifying the affected data subjects, the relevant
parties, the law enforcement agencies, the Commissioner, relevant regulators and such other
parties who may be able to take remedial actions as soon as practicable after the defection of the
data breach. For notifying the Commissioner, a “Data Breach Notification Form” can be used.
13. Direct marketing
The data user must:
inform the data subject (i) that the data user intends to so use the personal data; and (ii) that the
data user may not so use the data unless the data user has received the data subject’s consent to
5 | Data protection and cybersecurity laws in Hong Kong
the intended use – this “consent” needs to be “an indication of no objection to the use or
provision” and hence, silence or lack of response will not be deemed to be consent;
provide the data subject with the following information in relation to the intended use (i) the kinds
of personal data to be used; and (ii) the classes of marketing subjects in relation to which the data
is to be used – the description of such classes should be specific, making reference to the
distinctive features of the goods, facilities or services so that it is practicable for the customers to
ascertain the goods, facilities or services to be marketed with a reasonable degree of certainty;
and
provide the data subject with a channel through which the data subject may, without charge by the
data user, communicate the data subject’s consent to the intended use – a data user can only elect
a response channel that enables the data subject’s consent to be made in writing.
14. Cookies and adtech
There are no specific requirements in relation to use of cookies.
However, the use of cookies to collect personal data needs to be in compliance with Data Protection Principle
1(3) in Schedule 1 to the Ordinance that requires:
the data subject is explicitly or implicitly informed, on or before collecting the data, of (i) whether it
is obligatory or voluntary for him or her to supply the data; and (ii) where it is obligatory for him or
her to supply the data, the consequences for him or her if he or she fails to supply the data; and
he or she is explicitly informed: (i) on or before collecting the data, of (A) the purpose (in general or
specific terms) for which the data is to be used; and (B) the classes of persons to whom the data
may be transferred; and (ii) on or before first use of the data for the purpose for which it was
collected, of (A) his or her rights to request access to and to request the correction of the data;
and (B) the name or job title, and address, of the individual who is to handle any such request
made to the data user.
15. Risk scale
Moderate
16. Useful links
https://www.pcpd.org.hk/
Cybersecurity
1. Local cybersecurity laws and scope
The most significant laws that cover cybersecurity matters include provisions under:
Crimes Ordinance (Cap 200): (1) s.161 Access to computer with criminal or dishonest intent; and
(2) s.60 Destroying or damaging property;
6 | Data protection and cybersecurity laws in Hong Kong
s.27 A (unauthorised access to computer by telecommunications) under Telecommunications
Ordinance (Cap 106);
Control of Obscene and Indecent Articles Ordinance (Cap. 390);
Prevention of Child Pornography Ordinance (Cap 579); and
The Unsolicited Electronic Messages Ordinance (Cap 593)
2. Anticipated changes to local laws
There are no anticipated changes to local laws, although there has been more pressure to introduce laws
against doxing
3. Application
It mainly criminalises conduct around unauthorised access to computer and disseminating obscene, child
pornography and unsolicited electronic messages.
4. Authority
Information Commissioner’s Office www.ico.org.uk
The Cyber Security and Technology Crime Bureau (Hong Kong Police)
The Communications Authority (for reporting spam)Communications Authority - Home (coms-auth.hk)
5. Key obligations
N/A – There is no prescribed obligation imposed on cyber users or operators to adopt security measures
except those involving handling personal data as specified in Personal Data (Privacy) Ordinance (Cap 486)
(the “ Ordinance”)
6. Sanctions & non-compliance
Administr ative sanctions:
N/A
Crimina l sanctions:
Hong Kong Police will enforce the provisions of the relevant Ordinances. Penalties will range from a level 4
fine (HKD 25,000) to imprisonment for five years.
Oth ers:
7 | Data protection and cybersecurity laws in Hong Kong
N/A
7. Is there a national computer emergency response team (CERT) or
computer security incident response team (CSIRT)?
No
8. National cybersecurity incident management structure
N/A
9. Other cybersecurity initiatives
Hong Kong Monetary Authority has issued various non-binding cybersecurity guidelines for
authorised institutions such as Cyber Resilience Assessment Framework and cybersecurity
guidelines with respect to the use of stored value facilities, ebanking systems and artificial
intelligence.
Securities and Futures Commission has published guidelines and circulars such as the Guidelines
for Reducing and Mitigating Hacking Risks Associated with Internet Trading and specific guidelines
in relation to the use of external electronic data storage.
Insurance Authority has issued the Guideline on Cybersecurity laying down the minimum
cybersecurity standards that authorised insurers must observe.
The Commissioner for the Electronic Health Record has issued codes of practice regarding the use
of the electronic health record sharing system by healthcare providers to access and share
patients’ electronic health records.
The Office of the Government Chief Information Office has issued guidelines on cybersecurity
controls and measures applicable to various government offices and departments.
10. Useful links
NA
8 | Data protection and cybersecurity laws in Hong Kong
Authors
Key contacts
Jonathan Chu
Partner
Hong Kong (CMS CMNO - Lau,
Horton & Wise LLP)
Jonathan Chu
Hong Kong (CMS CMNO - Lau,
Horton & Wise LLP)
9 | Data protection and cybersecurity laws in Hong Kong'''

In [17]:
#Print keyword, category, entity and summary from the text (for PDF documents)
#metadata not included: Does not work for text (works only for URL, HTML)

authenticator = IAMAuthenticator('Suk_Yjz4CzP71YD4U_HcEGN0q76a24ZRODrz-rXVORPJ')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://api.kr-seo.natural-language-understanding.watson.cloud.ibm.com/instances/0de8d00e-0a41-4121-ac07-5264f62c77c2')

try: 
    response = natural_language_understanding.analyze(
        language = 'en',
        text= TEXT,
        features=Features(
            keywords=KeywordsOptions(limit=10),
            categories=CategoriesOptions(limit=3),
            entities=EntitiesOptions(limit=3))).get_result()

    print('[Keyword - top 10]')
    for i in range(len(response.get("keywords"))):
        print(response.get("keywords")[i].get("text"))
    print('')

    #Category - top 3
    print('[Category - top3]')
    for i in range(len(response.get("categories"))):
        print(response.get("categories")[i].get("label"))
    print('')


    #Entities 
    print('[Entities]')
    for i in range(len(response.get("entities"))):
        print(response.get("entities")[i].get("type") + ":", end = ' ')
        print(response.get("entities")[i].get("text"))
except:
    traceback.print_exc()    

[Keyword - top 10]
Data protection
Local data protection laws
Data Protection Officer
data users
Office of the Privacy Commissioner
Office of the Government Chief Information Office
Data subject rights
cybersecurity laws
Hong Kong
data user

[Category - top3]
/news and politics/law
/technology & computing/computing/information and network security
/technology & computing/computing/internet/it and internet support

[Entities]
Location: Hong Kong
JobTitle: Data Protection Officer
Organization: CMS Expert Guide


In [18]:
#Extract Abstract/summary part from the document URL using CSS Selector extension
from pyquery import PyQuery as pq
import requests

url = 'https://www.mondaq.com/hongkong/data-protection/1134772/hong-kong-issues-guidance-on-the-use-of-ai#:~:text=The%20Guidance%20further%20recommends%20that,irrelevant%20to%20the%20purposes%20concerned'
try:
    content = requests.get(url).content
    doc = pq(content)

    #input css path 
    respondent = doc('p:nth-child(7) , p:nth-child(6) , p:nth-child(5) , p:nth-child(4) , p:nth-child(3) , #articlebody p:nth-child(2)').text() 
    print("Abstract: ")
    print(respondent)
except:
    traceback.print_exc()

Abstract: 
Hong Kong now has its own set of recommended best practices for the development and use of AI published in a guidance note issued by the PCPD. Businesses which intend to or have begun to use AI in their operations are advised to consider the risk levels of their respective AI systems and implement the suggested measures for better protection of individual consumers. An artificial intelligence ("AI") system is a machine-based system which makes predictions, recommendations for decisions influencing real or virtual environments, based on a given set of human-defined objectives. The potential social and economic benefits of AI are significant. Healthy use of AI can drive innovation and improve efficiency in a wide range of fields. However, given the nature of what an AI system does, it also comes with potential risks which no business should ignore. AI systems often involve the profiling of individuals and the making of automated decisions which have real impact on human beings